In [2]:
p = 1331169830894825846283645180581
E = EllipticCurve(GF(p), [-35, 98])

# Generator:
G = E(479691812266187139164535778017, 568535594075310466177352868412)
# Alice's public key::
A = E(1110072782478160369250829345256, 800079550745409318906383650948)
# Bob's public key:
B = E(1290982289093010194550717223760, 762857612860564354370535420319)

In [3]:
# DLP: we want to find coefficient a such that aG = A
# we want to find the order of the curve E, as well as the points G, A, and B
# this restricts the space of possible secret keys
oE = E.order()
oG = G.order()
oA = A.order()

print("Order of curve E: " + str(oE))
print("Factorization of oE: " + str(oE.factor()))
print("Order of G: " + str(oG))
print("Factorization of oG: " + str(oG.factor()))
print("Order of A: " + str(oA))
print("Factorization of oA: " + str(oA.factor()))

# We can now decompose the DLP of the group into 2 DLPs of the subgroups via the Chinese Remainder Theorem:
# Secret key (a mod rs) can be found by finding (a mod r) and (a mod s) individually.

# We see that the largest prime shared by E, G, and A is
r = 1153763334005213
# Then we let the remainder (which is smooth) be denoted by
s = 2 * 7 * 271 * 23687
# From this, r*s*G = 0, so we can find, for E[r], sA = a_r(sG) (solving for a_r)
# and rA = a_s(rG) (solving for a_s)
# combining a_r and a_s using CRT should give a

sA = s * A
sG = s * G
print("The goal is to find some a_r such that " + str(sA) + " = a_r * " + str(sG))

# Both sA and sG fall in the r-torsion, as they are in the base subgroup. However, we need a subgroup not equal to the original base
# subgroup in order to compute a pairing.
# Therefore, we need to find the embedding degree k such that the field extension to k, F_{p^k} contains the entire r-torsion.
for k in range(1, 10):
    if (p^k - 1) % r == 0:
        print(k)
        break
# the embedding degree k is 2. This is the weakness in this particular curve we exploit in this attack and make the DLP easy.

# Now, we define a new curve E2 which is the extension of E on F_{p^2}:
E2 = E.change_ring(GF(p^2))
print(E2)


Order of curve E: 1331169830894823538756977170156
Factorization of oE: 2^2 * 7 * 271^2 * 23687^2 * 1153763334005213
Order of G: 103686954799254136375814
Factorization of oG: 2 * 7 * 271 * 23687 * 1153763334005213
Order of A: 103686954799254136375814
Factorization of oA: 2 * 7 * 271 * 23687 * 1153763334005213
The goal is to find some a_r such that (113155530967107107721855063813 : 1015480459109370604595193250002 : 1) = a_r * (513790355677961677339711089503 : 826383845826281865370403576638 : 1)
2
Elliptic Curve defined by y^2 = x^3 + 1331169830894825846283645180546*x + 98 over Finite Field in z2 of size 1331169830894825846283645180581^2


In [4]:
# Solving for a_s is much easier as s is smooth:
rA = r * A
rG = r * G
print("The goal is to find some a_s such that " + str(rA) + " = a_s * " + str(rG))

# This can be solved immediately using Sage's discrete log solver:
a_s = rG.discrete_log(rA)
print(a_s)

# we can verify that a_s is in fact the solution:
print("rA = " + str(rA))
print("a_s * rG = " + str(a_s*rG))

The goal is to find some a_s such that (995504861340942059004788910126 : 107940030991765253544851719980 : 1) = a_s * (1233781420613120848373297836636 : 470821891173910927420586962380 : 1)
32135505
rA = (995504861340942059004788910126 : 107940030991765253544851719980 : 1)
a_s * rG = (995504861340942059004788910126 : 107940030991765253544851719980 : 1)


In [7]:
# we cycle through random points defined on E2 until we find a point that lies in a non-base subgroup
p2 = E2.random_point()
print(p2)

(866186263619328858219966257965*z2 + 979185601001975661335500150128 : 740075209261918744062934693482*z2 + 509802657324424945341899090322 : 1)
2^4 * 7 * 11 * 29 * 191 * 271 * 457 * 919 * 1409 * 23687 * 1153763334005213
weil1: 189127879196204868929465099759*z2 + 233398305187926584987110651425
weil2: 45188818773932647637157215703*z2 + 179071977676504777171553799836
774386641791944
sA = (113155530967107107721855063813 : 1015480459109370604595193250002 : 1)
a_r * sG = (113155530967107107721855063813 : 1015480459109370604595193250002 : 1)


In [6]:
# let p2 = (882582545787169929420298882396*z2 + 493247577639681608912475745989 : 623574581147517257440197646780*z2 + 566741962044583632192586866049 : 1)
print(str(p2.order().factor()))
remainder = p2.order() / r
q2 = p2 * remainder # q2 is now the point we will use to compute pairings

# finally, we can compare the Weil Pairings of (sA, q2) and (sG, q2)
# in order to do this, we first must send sA and sG to the new curve E2
sA2 = E2(sA)
sG2 = E2(sG)
weil1 = q2.weil_pairing(sA2, r)
weil2 = q2.weil_pairing(sG2, r)
print("weil1: " + str(weil1))
print("weil2: " + str(weil2))

# then we can use sage's built-in discrete log solver:
a_r = weil1.log(weil2)
print(a_r)

# now, we can verify that a_r is in fact the solution to the DLP:
print("sA = " + str(sA))
print("a_r * sG = " + str(a_r*sG))
# these values should match

2^4 * 7 * 11 * 29 * 191 * 271 * 457 * 919 * 1409 * 23687 * 1153763334005213
weil1: 361708055255809691589369111058*z2 + 853704147612401762018651044800
weil2: 886701389105137955538030605814*z2 + 197959008355509951055314416601
774386641791944
sA = (113155530967107107721855063813 : 1015480459109370604595193250002 : 1)
a_r * sG = (113155530967107107721855063813 : 1015480459109370604595193250002 : 1)


In [10]:
# Now, we can use the Chinese Remainder Theorem to compute the actual value of a:
a = CRT(a_r, a_s, r, s)

# testing to make sure a is in fact the solution to A = aG:
print("A = " + str(A))
print("aG = " + str(a*G))

A = (1110072782478160369250829345256 : 800079550745409318906383650948 : 1)
aG = (1110072782478160369250829345256 : 800079550745409318906383650948 : 1)
